In [44]:
import pandas as pd
import re

import nltk

In [68]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kdrcn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kdrcn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kdrcn\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kdrcn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
path = "128.turkish/restaurants_train_turkish.xml.dat"

In [24]:
with open(path, "r", encoding="utf-8") as file:
    text = file.read()
    file.close()
data = pd.DataFrame({"Text": text.split("\n"), "Tag": "Text"})

In [25]:
sentiment_index = data[data["Text"].isin(["Positive", "Negative", "Neutral"])].index
data.loc[sentiment_index, "Tag"] = "Sentiment"
data.loc[sentiment_index-1, "Tag"] = "Aspect"
df = pd.DataFrame({"Phrase": data[data["Tag"] == "Text"]["Text"].reset_index(drop=True), 
                   "Aspect": data[data["Tag"] == "Aspect"]["Text"].reset_index(drop=True), 
                   "Sentiment": data[data["Tag"] == "Sentiment"]["Text"].reset_index(drop=True)})
df = df.drop(1385, axis=0)

In [69]:
df.head()

,Phrase,Aspect,Sentiment
0,$T$ sahane evet ama servis rezalet.,Manzara,Positive
1,Manzara sahane evet ama $T$ rezalet.,servis,Negative
2,"Soguk su isteyince, soguk yok, butun sulari di...",mezenin,Negative
3,"Soguk su isteyince, soguk yok, butun sulari di...",garson,Negative
4,"$T$ iyi hos, lezzetler iyi ama heyecan verici ...",Yemekler,Positive


In [49]:
train = df.copy()

In [70]:
train.head()

,Phrase,Aspect,Sentiment
0,t sahane evet servis rezalet,manzara,Positive
1,manzara sahane evet t rezalet,servis,Negative
2,soguk su isteyince soguk yok butun sulari disa...,mezenin,Negative
3,soguk su isteyince soguk yok butun sulari disa...,garson,Negative
4,t iyi ho lezzetler iyi heyecan verici bi taraf...,yemekler,Positive


In [ ]:
from nltk.corpus import stopwords
from textblob import Word

sw = set(stopwords.words("turkish"))

In [50]:
train["Phrase"] = train["Phrase"].apply(lambda x: " ".join(x.lower() for x in x.split()))
train["Aspect"] = train["Aspect"].apply(lambda x: " ".join(x.lower() for x in x.split()))

train["Phrase"] = train["Phrase"].apply(lambda x: " ".join(re.sub("[^\w\s]", "", x) for x in x.split()))
train["Aspect"] = train["Aspect"].apply(lambda x: " ".join(re.sub("[^\w\s]", "", x) for x in x.split()))

train["Phrase"] = train["Phrase"].apply(lambda x: " ".join(re.sub("\d", "", x) for x in x.split()))
train["Aspect"] = train["Aspect"].apply(lambda x: " ".join(re.sub("\d", "", x) for x in x.split()))

train["Phrase"] = train["Phrase"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

train["Phrase"] = train["Phrase"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [67]:
pd.DataFrame(" ".join(df["Phrase"]).split(), columns=["Kelimeler"]).value_counts()

Kelimeler    
$T$              1217
bir               467
ve                362
çok               266
ama               171
                 ... 
içerdeki            1
içeride             1
içerisi             1
içi,cumartesi       1
😎                   1
Length: 4230, dtype: int64